# 02 · Fine-tuning Eficiente con QLoRA

**Objetivo:** entrenar con QLoRA sobre un dataset pequeño (ej. Alpaca subset) y evaluar pérdidas.

**Índice:** 1) Setup 2) Dataset 3) Configuración QLoRA 4) Entrenamiento 5) Evaluación 6) Exportación

## 1. Setup

In [ ]:
# !pip install torch transformers datasets peft accelerate bitsandbytes trl
import os


## 2. Dataset

In [ ]:
# from datasets import load_dataset
# ds = load_dataset("tatsu-lab/alpaca")
# ds_small = ds['train'].select(range(1000))
# ds_small


## 3. Configuración QLoRA

In [ ]:
# from transformers import AutoModelForCausalLM, AutoTokenizer
# from peft import LoraConfig, get_peft_model
# MODEL_NAME = "<modelo-base>"
# tok = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=True)
# base = AutoModelForCausalLM.from_pretrained(MODEL_NAME, load_in_4bit=True, device_map="auto")
# lora_cfg = LoraConfig(r=64, lora_alpha=16, target_modules=["q_proj","v_proj"], lora_dropout=0.05, bias="none")
# model = get_peft_model(base, lora_cfg)
# model.print_trainable_parameters()


## 4. Entrenamiento (TRL / Trainer)

In [ ]:
# from transformers import TrainingArguments, Trainer
# def format_sample(example):
#     return {"text": f"Instrucción: {example['instruction']}\nEntrada: {example.get('input','')}\nRespuesta:"}
# # ds_fmt = ds_small.map(format_sample)
# training_args = TrainingArguments(
#     output_dir="./outputs/qlora",
#     per_device_train_batch_size=1,
#     gradient_accumulation_steps=8,
#     learning_rate=2e-4,
#     num_train_epochs=1,
#     logging_steps=10,
#     fp16=True
# )
# trainer = Trainer(model=model, args=training_args, train_dataset=None)  # TODO: ds_fmt
# # trainer.train()


## 5. Evaluación rápida

In [ ]:
# TODO: eval simple con prompts de sanity-check
# prompts = ["¿Qué es QLoRA?", "Explica PEFT en una frase."]
# for p in prompts:
#     pass


## 6. Exportación y carga de adaptadores

In [ ]:
# model.save_pretrained("outputs/qlora-adapter/")
# # Para cargar luego:
# # from peft import PeftModel
# # base = AutoModelForCausalLM.from_pretrained(MODEL_NAME, device_map="auto")
# # model = PeftModel.from_pretrained(base, "outputs/qlora-adapter/")


## Notas
- Ajusta `r`, `lora_alpha` y `target_modules` según el modelo.
- Controla VRAM con batch pequeño y acumulación de gradientes.